# 1 Train Yolo V8 using Pytorch & Ultralytics Framework
- Setting GPU Environment
- Installing Ultralytics YOLOv8 on Colab
- Download Dataset from Roboflow (previous notebook)
- Train Custom YOLOv8 Detector
- Infer Custom Objects with Saved YOLOv8 (`.pt`)

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%205/5.1 train-yolov8-object-detection-on-custom-dataset.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.1 Connect GPU Environment 

- Click `Connect` button in top right Google Colab notebook,<br>
<img src="resource/cl-connect-gpu.png" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="resource/cl-connect-gpu-success.png" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## 1.2 Installing Ultralytics YOLOv8 on Colab
- `Ultralytics YOLO` is python library that help simplify YOLO training model in `Pytorch`
- It's wrap `Pytorch` functionality as simple CLI command like we use `Darknet` framework
- More about `Ultralytics YOLO` (https://docs.ultralytics.com/)  

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

### 1.2.1 Ultralytics CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## 1.3 Download Dataset from Roboflow (previous notebook)

- Back to `Roboflow` > `Project` > `Versions` menu
- Then click `Download Dataset`<br>
<img src="resource/rb-download-dataset.png" width="850px">
- Choose `YOLO Darknet` format and select `Show download code` then click `Continue` <br>
<img src="resource/rb-download-format.png" width="350px">
- click `Copy` icon to copy roboflow download code<br>
<img src="resource/rb-copy-download-code.png" width="350px">
- Then replace below code using the copied roboflow download code above,


In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="xxxxxxxxxxxxxxxxxx")
# project = rf.workspace("xxxxxxxxxxxxxxxxxxx").project("xxxxxxxxxxxxxxxxxx")
# version = project.version(1)
# dataset = version.download("yolov8")


## Custom Training
- Don't forget to change `imgsz=` to image size uploaded in Roboflow
- Default `imgsz=320`

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=320 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

- Check Results Plot

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

- Check mAP on Validation Dataset

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Test inference on Custom Yolo V8 Model
- trained model located in `{HOME}/runs/detect/train/weights/best.pt`
- we will try to do inferencing using that model to detect test image in `{dataset.location}/test/images`

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.4 source={dataset.location}/test/images save=True

- Above command will generating detected image that located in `/content/runs/detect/`
- Show the detection image result using script below

In [ ]:
import glob
from IPython.display import Image, display

# Define the base path where the folders are located
base_path = '/content/runs/detect/'

# List all directories that start with 'predict' in the base path
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Find the latest folder by modification time
latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(f'{latest_folder}/*.jpg')[:3]

# Display each image
for image_path in image_paths:
    display(Image(filename=image_path, width=320))
    print("\n")

## Export to ONNX
- To use Pytorch (`.pt`) model generated by Utralytic in OpenCV DNN, we will need to convert that into `ONNX` format

In [ ]:
# Export the model
from ultralytics import YOLO

model = YOLO(HOME + "/runs/detect/train/weights/best.pt")  # Load a model
model.export(format="onnx", opset=12, imgsz=[320, 320])

In [ ]:
# Download Yolo Model.
from google.colab import files
import shutil

shutil.copy(HOME + "/runs/detect/train/weights/best.onnx", HOME + "/yolov8.onnx")
files.download(HOME + "/yolov8.onnx")

________________________________________________________________________

# Source
- https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb